# Worksheet 08

Name: Kian Boon Tan

UID: U93983891

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.181804983495415, 3.8416785411107113, 4.348880909523758, 4.586677271515826, 4.722916736639092, 5.9890359011841126, 4.4500887536213085, 4.44944387078011, 3.255062806782293, 5.4361520877714495]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[8.884843056660241, 9.1492059153979, 9.232973770945886, 8.56077146100575, 6.864235912248607, 7.753310478020193, 7.715720141629531, 6.6728555580344, 5.196441766405652, 6.840267039369606]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.4361520877714495, 6.840267039369606, 5.196441766405652, 3.255062806782293, 4.44944387078011, 4.4500887536213085, 6.6728555580344, 5.9890359011841126, 4.722916736639092, 4.586677271515826]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

$S_1 \sim N(5, 1), P(S_1) = 0.5$

$S_2 \sim N(8, 1), P(S_2) = 0.5$

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[6.840267039369606, 6.6728555580344, 5.9890359011841126]
[5.4361520877714495, 5.196441766405652, 3.255062806782293, 4.44944387078011, 4.4500887536213085, 4.722916736639092, 4.586677271515826]
P(S_1) = 0.3,  P(S_2) = 0.7
mean_1 = 6.500719499529374,  mean_2 = 4.585254756216534
var_1 = 0.13558115308825155,  var_2 = 0.4175219631250129


The parameter estimates given do not seem very close to the true values.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j] ** 0.5))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.4361520877714495
probability of observing that point if it came from cluster 0 =  0.016584683308646035
probability of observing that point if it came from cluster 1 =  0.25942667370934713
point =  6.840267039369606
probability of observing that point if it came from cluster 0 =  0.7082025544118888
probability of observing that point if it came from cluster 1 =  0.001399240400640685
point =  5.196441766405652
probability of observing that point if it came from cluster 0 =  0.0020429516216847687
probability of observing that point if it came from cluster 1 =  0.39472313670634734
point =  3.255062806782293
probability of observing that point if it came from cluster 0 =  1.455658622300351e-17
probability of observing that point if it came from cluster 1 =  0.07418653338042541
point =  4.44944387078011
probability of observing that point if it came from cluster 0 =  1.9755718897922683e-07
probability of observing that point if it came from cluster 1 =  0.6039175218601756
point = 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_s = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.27683552864224087,  P(S_2) = 0.7231644713577591
mean_1 = 6.5341935427709705,  mean_2 = 4.633796858486544
var_1 = 0.13436257676842145,  var_2 = 0.4746074704999107


#### Original Values:
```
P(S_1) = 0.3,  P(S_2) = 0.7
mean_1 = 6.500719499529374,  mean_2 = 4.585254756216534
var_1 = 0.13558115308825155,  var_2 = 0.4175219631250129
```

The means and variances appear to be slowly moving towards the correct values, but not the probabilities.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
# Recalculate probabilities based on new estimates.
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j] ** 0.5))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  5.4361520877714495
probability of observing that point if it came from cluster 0 =  0.01225215075832808
probability of observing that point if it came from cluster 1 =  0.293898105626826
point =  6.840267039369606
probability of observing that point if it came from cluster 0 =  0.7680161591825406
probability of observing that point if it came from cluster 1 =  0.003429669153009612
point =  5.196441766405652
probability of observing that point if it came from cluster 0 =  0.0013950162873206978
probability of observing that point if it came from cluster 1 =  0.4148611585562654
point =  3.255062806782293
probability of observing that point if it came from cluster 0 =  4.560626310236218e-18
probability of observing that point if it came from cluster 1 =  0.07816641481142488
point =  4.44944387078011
probability of observing that point if it came from cluster 0 =  1.0298689133651598e-07
probability of observing that point if it came from cluster 1 =  0.5587186253815748
point =  4.4

Most of the points have the same relative ratio, the points which the algorithm were sure of previously remain the same. For point 8 however, it appears that the algorithm is slowly favoring it coming from S2 over S1.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [13]:
# Assign a hard assignment to each point based on which relative probability is higher.
probs = zip(data, prob_s0_x, prob_s1_x)

for p in probs:
    print(f"Point {p[0]} belongs to cluster: ", end="")
    
    if p[1] > p[2]:
        print("0")
        
    else:
        print("1")

Point 5.4361520877714495 belongs to cluster: 1
Point 6.840267039369606 belongs to cluster: 0
Point 5.196441766405652 belongs to cluster: 1
Point 3.255062806782293 belongs to cluster: 1
Point 4.44944387078011 belongs to cluster: 1
Point 4.4500887536213085 belongs to cluster: 1
Point 6.6728555580344 belongs to cluster: 0
Point 5.9890359011841126 belongs to cluster: 0
Point 4.722916736639092 belongs to cluster: 1
Point 4.586677271515826 belongs to cluster: 1
